In [ ]:
#Mini ATM Machine
# Combining OOP principles in a real-world project
# Designing a modular class structure
# Implementing Secure User Authentication
# Managing Account transactions

In [ ]:
# Mini ATM Machine will allow users to
# Authenticate with PINs securely.
# Check account balance
# Deposit money
# Withdraw money with balance validation
# Change PIN 
# Exit securely

# Classes overview :
## BankAccount : account_number, pin, balance
### Methods : check_balance(), deposit(), withdraw(), change_pint()

## ATM
### Manages account authentification
### Provides the main menu for users

# Concepts Applied:
## Encapsulation : Secure PIN handling and balance access.
## Static Method: For utility tasks like PIN validation
## Class Method: to maintain account-level settings
## Polymorphism: Flexibility in transation operations

In [2]:
# Mini ATM Machine
# Bonus challenge : Add an admin section, allow multiple accounts per user, add a transaction log

class BankAccount:
    def __init__(self, username, account_number, pin, balance=0):
        self.username = username
        self.account_number = account_number
        self.__pin = pin
        self.__balance = balance

    #Validate PIN
    def validate_pin(self, entered_pin):
        return entered_pin == self.__pin
    
    #Get PIN
    def get_pin(self):
        return self.__pin
    
    #Get Balance
    def get_balance(self):
        return self.__balance
    
    # Check Balance
    def check_balance(self):
        print(f"Current Balance: {self.__balance}")

    # Deposit Money
    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            print(f"Deposited {amount}. New Balance: {self.__balance}")
        else:
            print("Invalid deposit amount.")

    # Withdraw Moneuy
    def withdraw(self, amount):
        if amount > self.__balance:
            print("Insufficient funds.")
        elif amount > 0:
            self.__balance -= amount
            print(f"Withdrew {amount}. New Balance: {self.__balance}")
        else:
            print("Invalid withdrawal amount")

    # Change PIN
    def change_pin(self, old_pin, new_pin):
        if self.validate_pin(old_pin) and len(new_pin) == 4 and new_pin.isdigit():
            self.__pin = new_pin
            print("PIN changed successfully.")
        else:
            print("Failed to change PIN. Ensure the old PIN is correct and the new PIN is 4 digits.")

import json
import csv
from datetime import datetime

class User:
    def __init__(self, username, password):
        self.username = username
        self.__password = password
        self.accounts = {}

    def get_password(self):
        return self.__password
    
    # Create Account
    def create_account(self):
        account_number = input("Enter account number: ").strip()
        pin = input("Set a 4-digit PIN: ")
        if len(pin) == 4 and pin.isdigit():
            self.accounts[account_number] = BankAccount(self.username, account_number, pin)
            ATM.save_transaction_log(self.username, "Create Account", self.accounts[account_number])
            print("Account created successfully.")
        else:
            print("Invalid PIN. PIN must be 4-digit")

    # Authenticate Account
    def authenticate_account(self):
        account_number = input("Enter account number: ").strip()
        pin = input("Enter PIN: ")

        account = self.accounts.get(account_number)
        if account and account.validate_pin(pin):
            print("Autentification Successful")
            ATM.save_transaction_log(self.username, "Authenticate", self.accounts[account_number])
            self.account_menu(account)
            return account
        else:
            print("Invalid account number or PIN.")
            return None
        
    # User Menu
    def user_menu(self, user_list=None):
        while True:
            print("\n--- ATM Menu ---")
            menu_indx = 0 or len(self.accounts)
            accounts = list(self.accounts.values())
            print("1. Create account")
            if self.accounts:
                for indx, account in enumerate(accounts):
                    print(f"{indx + 2}. Access account {account.account_number}")
            print(f"{menu_indx + 2}. Exit")

            choice = int(input(f"Enter your choice (1-{menu_indx + 2}): ").strip())

            if choice == 1:
                self.create_account()
            elif choice >= 2 and choice <= menu_indx + 1:
                self.account_menu(list(self.accounts.values())[choice - 2])
            elif choice == menu_indx + 2:
                print("Exiting user menu.")
                break
            else:
                print(f"Invalid choice. Please enter a number between 1 and {len(self.accounts) + 2}.")

    # Account Menu
    def account_menu(self, account):
        while True:
            print("\n--- ATM Menu ---")
            print("1. Check Balance")
            print("2. Deposit")
            print("3. Withdraw")
            print("4. Change PIN")
            print("5. Go back to User Menue")
            
            choice = input("Enter your choice (1-5): ")

            if choice == "1":
                account.check_balance()
                ATM.save_transaction_log(self.username, "Check Balance", account)
            elif choice == "2":
                amount = float(input("Enter deposit amount: "))
                account.deposit(amount)
                ATM.save_transaction_log(self.username, "Deposit", account, amount)
            elif choice == "3":
                amount = float(input("Enter amount to withdraw: "))
                account.withdraw(amount)
                ATM.save_transaction_log(self.username, "Withdraw", account, amount)
            elif choice == "4":
                old_pin = input("Enter old PIN : ").strip()
                new_pin = input("Enter new PIN : ").strip()
                account.change_pin(old_pin, new_pin)
                ATM.save_transaction_log(self.username, "Change PIN", account)
            elif choice == "5":
                print("Go back to user menu.")
                break
            else:
                print("Invalid choice. Please enter a number between 1 and 5.")

    def get_user_data(self):
        return {
            "username": self.username,
            "password": self.get_password(),
            "accounts": [
                {
                    "account_number": account.account_number,
                    "pin": account.get_pin(),
                    "balance": account.get_balance()
                }
                for _, account in self.accounts.items()
            ]
        }
        
        
class Admin(User):
    def __init__(self, username, password):
        self.username = username
        self.__password = password

    def get_password(self):
        return self.__password

    # Admin Menu
    def user_menu(self, user_list=None):
        while True:
            print("\n--- Admin Menu ---")
            print("1. Create user")
            print("2. Delete user")
            print("3. Logout")

            choice = input("Enter your choice (1-3) : ").strip()

            if choice == "1":
                return Admin.create_user()
            elif choice == "2":
                return Admin.delete_user(user_list)
            elif choice == "3":
                print("Admin logout successful. Go back to ATM Main Page.")
                return
            else:
                print("Invalid choice. Please enter a number between 1 and 3.")


    # Get user data for log purpose
    def get_user_data(self):
        return {
            "username": self.username,
            "password": self.get_password()
        }

    @staticmethod
    def create_user():
        while True:
            print("\n--- Mini ATM Menu ---")
            print("1. Create a regular user")
            print("2. Create an Admin user")
            print("3. Exit")

            choice = input("Enter your choice (1-3) : ")

            if choice == "2":
                super_admin = input("Enter Super Admin Password: ")

            if choice == "1" or (choice == "2" and ATM.check_super_admin_password(super_admin)):
                username = input("Enter new user username : ")
                password = input("Enter new user password : ")
                user = User(username, password) if choice == "1" else Admin(username, password)
                return user
            elif choice == "3":
                print("Exiting User Creation menu.")
                break
            else:
                print("Invalid choice. Please try again.")
            
    @staticmethod
    def delete_user(user_dict):
        if not user_dict:
            print("No user to delete.")
        while True:
            print("\n---- User Deletion Menu ----")
            users = list(user_dict.values())
            for indx, user in enumerate(users):
                print(f"{indx + 1}. {user.username}")
            print(f"{len(users) + 1}. Exit")
            
            choice = int(input(f"Enter your choice (1-{len(users) + 1}) : ").strip())

            if choice >= 1 and choice <= len(users):
                user = list(users.values())[choice - 1]
                validate_deletion = input(f"You are about to delete user '{user.username}' and their {len(user.accounts)}. Are you sure (yes/no) ? ").strip().lower()

                if validate_deletion == "yes":
                    validate_deletion.pop(choice - 1)
                    ATM.save_transaction_log(user.username, "User deleted")
                    print(f"User '{user.username}' successfully deleted.")
                else:
                    print(f"Cancel the deletion of user '{user.username}'")
            elif choice == len(user_dict) + 1:
                print("Exit User Deletion Menu.")
            break
    

class ATM:
    atm_database_file = './assets/atm.json'
    atm_transaction_log = './assets/atm_transaction_log.csv'
    super_administrator_password = "Sup3rAdm!n"

    def __init__(self):
        self.users = ATM.load_atm()

    @classmethod
    def check_super_admin_password(cls, password):
        return cls.super_administrator_password == password

    # Authenticate user
    def login_user(self):
        if not self.users:
            print("No registered user for ATM.")
            return
        username = input("Enter username for login : ").strip()
        password = input("Enter password for login : ").strip()

        if username in self.users and self.users[username].get_password() == password:
            ATM.save_transaction_log(username, "Login")
            user = self.users[username]
            new_user = user.user_menu(self.users)
            if new_user:
                self.users[new_user.username] = new_user
                print(f"{new_user.__class__.__name__ if new_user.__class__.__name__ == 'Admin' else 'Regular'} user created")
                ATM.save_transaction_log(new_user.username, f"{new_user.__class__.__name__ if new_user.__class__.__name__ == 'Admin' else 'Regular'} user created")
            self.save_atm()
            return
        print("Incorrect username or password. Please try again.")

    # Create user
    def create_user(self):
        attempts = 0
        while True:
            admin_users = [user for _, user in self.users.items() if isinstance(user, Admin)]
            username = None
            password = None

            while attempts < 3:
                if len(admin_users) == 0:
                    print("No admin user found. Please consider creating an Admin User. Using Super Admin credentials...")
                    password = input("Enter Super Admin Password : ").strip()
                else:
                    username = input("Enter Admin username : ").strip()
                    password = input("Enter Admin password : ").strip()

                if (password == ATM.super_administrator_password) or (username is not None and username in admin_users and admin_users[username] == password):
                    user = Admin.create_user()
                    if user:
                            self.users[user.username] = user
                            print(f"{user.__class__.__name__ if user.__class__.__name__ == 'Admin' else 'Regular'} user created")
                            ATM.save_transaction_log("SuperAdmin" if not username else username, "Create User")
                            self.save_atm()
                            break
                    else:
                        attempts += 1
            if attempts >= 3 and not user:
                print("Failed to create user. Please try again later.")
                break
            elif user:
                print("Exit User Creation...")
                break

    # Load transaction log content
    @staticmethod
    def load_transaction_log():
        try:
            with open(ATM.atm_transaction_log, 'r') as csv_file:
                return [row for row in csv.DictReader(csv_file)]
        except FileNotFoundError:
            print("No transaction log file found.")
        except Exception as e:
            print(f"An unexpected error occurred : {e}")
        return []

    # Define log line
    @staticmethod
    def log_line(username, operation, account=None, amount=None):
        return {
            "datetime": datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S"),
            "username": username,
            "account_number": account.account_number if account else "NA",
            "operation": operation,
            "amount": amount if amount else "NA",
            "balance_after_operation": account.get_balance() if account else "NA"
        }

    # Save transaction log
    @staticmethod
    def save_transaction_log(username, operation, account=None, amount=None):
        log_headers = ["datetime", "username", "account_number", "operation", "amount", "balance_after_operation"]
        try:
            transaction_log_content = ATM.load_transaction_log()
            transaction_log_content.append(ATM.log_line(username, operation, account, amount))
            with open(ATM.atm_transaction_log, 'w', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=log_headers)
                writer.writeheader()
                writer.writerows(transaction_log_content)
        except Exception as e:
            print(f"An unexpected error occurred : {e}")

    # Load Current State of ATM
    @staticmethod
    def load_atm():
        try:
            users = {}
            with open(ATM.atm_database_file, 'r') as json_file:
                atm_content = json.load(json_file)
                if atm_content:
                    for user_data in atm_content:
                        if "accounts" in user_data:
                            users[user_data["username"]] = User(user_data["username"], user_data["password"])
                            user = users[user_data["username"]]
                            for account_data in user_data["accounts"]:
                                user.accounts[account_data["account_number"]] = BankAccount(user_data["username"], account_data["account_number"], account_data["pin"], account_data["balance"])
                        else:
                            users[user_data["username"]] = Admin(user_data["username"], user_data["password"])
        except FileNotFoundError:
            print("No accounts available.")
        except Exception as e:
            print(f"An unexpected error occurred : {e}")
        return users

    # Convert accounts to data
    def convert_accounts(self):
        return [
            user.get_user_data()
            for _, user in self.users.items()
        ]

    # Save Current State of ATM
    def save_atm(self):
        try:
            with open(ATM.atm_database_file, 'w', newline='') as json_file:
                json.dump(self.convert_accounts(), json_file, indent=2)
        except Exception as e:
            print(f"An unexpected error occurred : {e}")


    # Main menu
    def main_menu(self):
        while True:
            print("\n--- Welcome to Mini ATM Machine ---")
            print("1. Login")
            print("2. Create a user")
            print("3. Exit")

            choice = input("Choose an option (1-3): ")

            if choice == "1":
                self.login_user()
            elif choice == "2":
                self.create_user()
            elif choice == "3":
                print("Thank you for using Mini ATM Machine. Goodbye !")
                break
            else:
                print("Invalid choice. Please enter a number between 1 and 3.")

# Start the ATM System
if __name__ == "__main__":
    atm = ATM()
    atm.main_menu()

Withdrew 150.0. New Balance: 23350.0

--- ATM Menu ---
1. Check Balance
2. Deposit
3. Withdraw
4. Change PIN
5. Go back to User Menue
Go back to user menu.

--- ATM Menu ---
1. Create account
2. Access account 65543
3. Access account 95688
4. Exit
Exiting user menu.

--- Welcome to Mini ATM Machine ---
1. Login
2. Create a user
3. Exit
Thank you for using Mini ATM Machine. Goodbye !
